In [1]:
# This is a Python script designed to run in a Google Colab notebook.
# To use, copy this code into a Colab cell and run it.

# Install required libraries (uncomment if running in Colab)
# !pip install matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random
from heapq import heappush, heappop

# Grid and simulation parameters
GRID_SIZE = 32
OBSTACLE_RATIO = 0.2  # 20% of grid cells are obstacles
FRAMES = 50  # Number of animation frames

# Initialize grid, agent, and goal
def setup():
    global grid, agent_pos, goal_pos, phi
    # Initialize grid: 0 = free, 1 = obstacle
    grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
    num_obstacles = int(GRID_SIZE * GRID_SIZE * OBSTACLE_RATIO)
    obstacle_indices = random.sample([(i, j) for i in range(GRID_SIZE) for j in range(GRID_SIZE)], num_obstacles)
    for i, j in obstacle_indices:
        grid[i, j] = 1

    # Set agent and goal positions (ensure they're not on obstacles)
    while True:
        agent_pos = [random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)]
        if grid[agent_pos[0], agent_pos[1]] == 0:
            break
    while True:
        goal_pos = [random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)]
        if grid[goal_pos[0], goal_pos[1]] == 0 and goal_pos != agent_pos:
            break

    # Initial latent representation Φ: [goal_x, goal_y, direction_x, direction_y]
    phi = np.array([goal_pos[0], goal_pos[1], 0.0, 0.0], dtype=float)
    return grid, agent_pos, goal_pos, phi

def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def a_star(start, goal):
    """A* pathfinding to simulate trajectory sampling."""
    open_list = [(0, start, [])]
    closed = set()
    while open_list:
        f_score, current, current_path = heappop(open_list)
        if current == goal:
            return current_path
        if current in closed:
            continue
        closed.add(current)
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            next_pos = (current[0] + dx, current[1] + dy)
            if (0 <= next_pos[0] < GRID_SIZE and 0 <= next_pos[1] < GRID_SIZE and
                grid[next_pos[0], next_pos[1]] == 0 and next_pos not in closed):
                g_score = len(current_path) + 1
                h_score = manhattan_distance(next_pos, goal)
                f_score = g_score + h_score
                heappush(open_list, (f_score, next_pos, current_path + [next_pos]))
    return []

def simulate_trajectory(phi):
    """Simulate a trajectory Z from latent Φ using A* as a proxy for p(Z|Φ)."""
    goal = (int(phi[0]), int(phi[1]))
    return a_star(tuple(agent_pos), goal)

def decode_trajectory(Z):
    """Decode trajectory Z to predicted states Ψ̂ (grid positions)."""
    return [(x, y) for x, y in Z]

def update_latent(Z, phi):
    """Update latent Φ using amortized recognition q(Φ|Z)."""
    if Z:
        last_pos = Z[-1]
        direction = np.array([last_pos[0] - agent_pos[0], last_pos[1] - agent_pos[1]], dtype=float)
        direction = direction / (np.linalg.norm(direction) + 1e-6) if np.linalg.norm(direction) > 0 else np.array([0.0, 0.0])
        return np.array([phi[0], phi[1], direction[0], direction[1]], dtype=float)
    return phi

# Animation update function
def update(frame):
    global agent_pos, phi, path
    # Simulate trajectory and update state
    Z = simulate_trajectory(phi)
    path = decode_trajectory(Z)
    if path:
        next_pos = path[0]
        agent_pos = [next_pos[0], next_pos[1]]
        path = path[1:]
        phi = update_latent(path, phi)

    # Clear the plot
    ax.clear()
    # Draw grid
    ax.imshow(grid, cmap='binary', origin='upper')
    # Draw path
    if path:
        path_x, path_y = zip(*path)
        ax.plot(path_y, path_x, 'b.', markersize=10, label='Path')
    # Draw agent and goal
    ax.plot(agent_pos[1], agent_pos[0], 'r^', markersize=15, label='Agent')
    ax.plot(goal_pos[1], goal_pos[0], 'g*', markersize=15, label='Goal')
    ax.set_title(f"Frame {frame+1}/{FRAMES}")
    ax.legend()
    ax.set_xticks(np.arange(0, GRID_SIZE, 1))
    ax.set_yticks(np.arange(0, GRID_SIZE, 1))
    ax.grid(True)
    return ax

# Set up the simulation
grid, agent_pos, goal_pos, phi = setup()

# Create the animation
fig, ax = plt.subplots(figsize=(8, 8))
ani = FuncAnimation(fig, update, frames=FRAMES, interval=200, repeat=False)
plt.close()  # Prevent duplicate display in Colab
from IPython.display import HTML
HTML(ani.to_jshtml())  # Display animation in Colab

Output hidden; open in https://colab.research.google.com to view.